In [1]:
import jtap_mice
jtap_mice.set_jaxcache()
from jtap_mice.inference import run_jtap, run_parallel_jtap, JTAPMiceData
from jtap_mice.viz import rerun_jtap_stimulus, rerun_jtap_single_run, jtap_plot_rg_lines, red_green_viz_notebook, plot_proposal_direction_outlier_pdf, draw_stimulus_image
from jtap_mice.utils import load_left_right_stimulus, JTAPMiceStimulus, ChexModelInput, d2r, i_, f_, slice_pt, init_step_concat, discrete_obs_to_rgb, stack_pytrees, concat_pytrees
from jtap_mice.evaluation import jtap_compute_beliefs, JTAPMice_Beliefs

import time
import rerun as rr
import genjax
from genjax import gen, ChoiceMapBuilder as C
import jax
import jax.numpy as jnp
from jax.debug import print as jprint
import numpy as np
from tqdm import tqdm
import jax.tree_util as jtu
from functools import partial
from matplotlib import pyplot as plt
from typing import List, Dict, Any, Tuple, NamedTuple

PIXEL_DENSITY = 10
SKIP_T = 4

In [2]:
LR_CONFIG= 'lr_v1'
LR_TRIAL = 1

stimulus_path = f'/home/arijitdasgupta/jtap-mice/notebooks/left_right_task/{LR_CONFIG}.json'

jtap_stimulus = load_left_right_stimulus(stimulus_path, pixel_density = PIXEL_DENSITY, skip_t = SKIP_T, trial_number = LR_TRIAL)

rgb_video_highres = load_left_right_stimulus(stimulus_path, pixel_density = PIXEL_DENSITY*5, skip_t = SKIP_T, rgb_only = True, trial_number = LR_TRIAL)

In [3]:
rerun_jtap_stimulus(discrete_obs = jtap_stimulus.discrete_obs, stimulus_name = "rg_stim1_discrete")

In [4]:
# Parameters based on default values in ChexModelInput @datastrucs.py

direction_flip_prob = 0.025

Model_Input = ChexModelInput(
    model_outlier_prob = 0.05,
    proposal_direction_outlier_tau = d2r(40.),
    proposal_direction_outlier_alpha = 3.5,
    σ_pos=10000.0,
    σ_speed=0.075,
    model_direction_flip_prob=direction_flip_prob,
    pixel_corruption_prob=0.01,
    tile_size=3,
    σ_pixel_spatial=1.0,
    image_power_beta=0.005,
    max_speed=1.0,
    max_num_occ=5,
    num_x_grid=8,
    num_y_grid=8,
    grid_size_bounds=(0.05, 0.95),
    simulate_every=1,
    σ_pos_sim=0.05,
    σ_speed_sim=0.075,
    σ_direction_sim=d2r(5.0),
    σ_pos_initprop=0.02,
    σ_direction_stepprop_flip_prob=direction_flip_prob,
    σ_speed_stepprop=0.3,
    σ_pos_stepprop=0.01
)
# PREPARE INPUT
Model_Input.prepare_hyperparameters()
Model_Input.prepare_scene_geometry(jtap_stimulus)

ESS_proportion = 0.09
smc_key_seed = np.random.randint(0, 1000000)
num_particles = 50

In [6]:
num_jtap_runs = 50

start_time = time.time()
JTAPMICE_DATA, xx = run_parallel_jtap(num_jtap_runs, smc_key_seed, Model_Input, ESS_proportion, jtap_stimulus, num_particles)
end_time = time.time()
mean_ESS = np.mean(JTAPMICE_DATA.inference.ESS.mean(axis=1))
print(f"Mean ESS: {100 * mean_ESS / num_particles:.1f}% of {num_particles} particles")
resampled = JTAPMICE_DATA.inference.resampled
resampled_pct = 100 * np.mean(resampled)
print(f"Resampling occurred in {resampled_pct:.1f}% of {jtap_stimulus.num_frames} frames")
print(f"Time taken for parallel JTAP: {end_time - start_time} seconds")

Mean ESS: 20.7% of 50 particles
Resampling occurred in 28.4% of 181 frames
Time taken for parallel JTAP: 1.4455361366271973 seconds


In [7]:
JTAPMICE_DATA

JTAPMiceData(num_jtap_runs=50, inference=JTAPMiceInference(tracking=TrackingData(x=array([[[ 8.98983553e-02,  8.15684423e-02,  1.09423414e-01, ...,
          1.01534568e-01,  1.27246067e-01,  1.09729014e-01],
        [ 1.90238997e-01,  2.18424737e-01,  2.13012189e-01, ...,
          1.92487165e-01,  1.93782464e-01,  2.02656984e-01],
        [ 1.90389380e-01,  1.77704632e-01,  1.95042938e-01, ...,
          2.15495840e-01,  1.87804013e-01,  2.01168001e-01],
        ...,
        [ 9.96508449e-02,  1.24437563e-01,  1.27907157e-01, ...,
          9.73544642e-02,  1.27501845e-01,  1.07279807e-01],
        [ 7.14113284e-03,  2.34959070e-02,  1.14064869e-02, ...,
          4.42587351e-03,  1.61537202e-04,  8.78445245e-03],
        [ 4.07199701e-03,  4.07199701e-03,  4.07199701e-03, ...,
          7.85461627e-03,  4.07199701e-03,  4.07199701e-03]],

       [[ 8.54195505e-02,  9.77846831e-02,  1.32112488e-01, ...,
          1.54940560e-01,  1.08545177e-01,  1.21192634e-01],
        [ 1.92354739e-01,  2.11475953e-01,  1.76851630e-01, ...,
          1.73722133e-01,  2.28476018e-01,  2.00757712e-01],
        [ 2.28389442e-01,  2.07921639e-01,  2.07721919e-01, ...,
          1.77198216e-01,  2.21503183e-01,  1.94772273e-01],
        ...,
        [ 9.07756686e-02,  1.24641754e-01,  7.77743906e-02, ...,
          9.63646919e-02,  1.25929549e-01,  1.17334232e-01],
        [ 1.73206888e-02,  2.25230549e-02,  2.25230549e-02, ...,
          2.25230549e-02,  2.25230549e-02,  2.25230549e-02],
        [ 9.24351159e-03,  1.94150507e-02,  1.15856398e-02, ...,
          7.69654708e-03,  4.55796067e-03,  1.13479504e-02]],

       [[ 8.86318311e-02,  9.94451195e-02,  1.09965056e-01, ...,
          1.05653480e-01,  1.21750914e-01,  8.34022462e-02],
        [ 1.77733764e-01,  2.26049915e-01,  1.87483147e-01, ...,
          1.96160048e-01,  2.03347325e-01,  1.97901264e-01],
        [ 1.94990337e-01,  2.17001334e-01,  1.78438127e-01, ...,
          2.15711534e-01,  2.26642892e-01,  2.27907345e-01],
        ...,
        [ 7.48546496e-02,  1.25962615e-01,  8.97203833e-02, ...,
          1.16367079e-01,  8.04474503e-02,  1.17016584e-01],
        [-1.34673575e-03,  2.47547626e-02,  2.62142532e-02, ...,
          4.88152076e-03,  1.97245404e-02,  2.36220211e-02],
        [ 1.28924800e-02,  1.39493933e-02,  2.58122738e-02, ...,
          2.58122738e-02,  2.58122738e-02,  1.39493933e-02]],

       ...,

       [[ 1.12132102e-01,  9.57273915e-02,  7.33702108e-02, ...,
          9.29347351e-02,  1.30109817e-01,  1.05620749e-01],
        [ 2.25018457e-01,  2.11059496e-01,  1.86400101e-01, ...,
          1.73611790e-01,  2.17658654e-01,  2.10637406e-01],
        [ 2.20028222e-01,  1.79974094e-01,  1.91306829e-01, ...,
          2.25859761e-01,  1.96800753e-01,  2.21482366e-01],
        ...,
        [ 8.67086872e-02,  1.08101599e-01,  1.20257363e-01, ...,
          9.07071680e-02,  1.17549039e-01,  9.82123017e-02],
        [ 2.21573142e-03,  2.21573142e-03,  7.18484074e-03, ...,
          2.24332511e-02,  2.21573142e-03,  2.21573142e-03],
        [ 8.78474675e-03,  2.43277028e-02,  1.24064358e-02, ...,
          3.69849987e-03,  1.80310123e-02,  2.52820700e-02]],

       [[ 1.13613725e-01,  1.03873953e-01,  9.10776034e-02, ...,
          1.07838728e-01,  7.58415237e-02,  1.04635321e-01],
        [ 1.86837390e-01,  1.72005624e-01,  1.95503235e-01, ...,
          2.18192711e-01,  1.89539775e-01,  1.98844731e-01],
        [ 2.19145119e-01,  2.17977047e-01,  1.91292509e-01, ...,
          2.02120632e-01,  2.24588320e-01,  2.15993509e-01],
        ...,
        [ 8.14656317e-02,  1.23348564e-01,  9.98880118e-02, ...,
          1.20114893e-01,  1.27179638e-01,  1.00983173e-01],
        [ 1.13090640e-02,  8.25096760e-03,  1.41584706e-02, ...,
          1.24673340e-02,  1.79518722e-02,  3.33145261e-03],
        [ 2.56294776e-02,  3.60669568e-03,  3.60669568e-03, ...,
          3.60669568e-03,  1.52375922e-02,  2.56294776e-02]],

       [[ 9.60516110e-02,  8.97298381e-02,  1.144

In [ ]:
JTAPMice_Beliefs = jtap_compute_beliefs(JTAPMICE_DATA)
jtap_run_idx = None
show_all_lines = False
show_std_band = True
jtap_plot_rg_lines(JTAPMice_Beliefs, stimulus = jtap_stimulus, show = "model", include_baselines=True, remove_legend=True, show_std_band=show_std_band, jtap_run_idx = jtap_run_idx, include_start_frame=True, show_all_beliefs=show_all_lines, plot_stat = "median", include_stimulus=True)

In [ ]:
DECISION_MODEL_VERSION = "v4"

jtap_decision_model_hyperparams = JTAP_Decision_Model_Hyperparams(
    key_seed=123,
    pseudo_participant_multiplier=50,
    press_thresh_hyperparams=(0.5253067868385957, 0.030245679028864637, 0.0, 1.0),
    tau_press_hyperparams=(1.0, 0.01, np.arange(30)),
    hysteresis_delay_hyperparams=None,
    regular_delay_hyperparams=(4.0, 1.2362500260780824, np.arange(30)),
    starting_delay_hyperparams=(11.0, 3.794423573053696, np.arange(30))
)

jtap_decisions, jtap_decision_model_params = jtap_compute_decisions(
    JTAPMice_Beliefs,
    jtap_decision_model_hyperparams,
    decision_model_version=DECISION_MODEL_VERSION
)
# jtap_metrics = jtap_compute_decision_metrics(
#     jtap_decisions,
#     jtap_stimulus,
#     partial_occlusion_in_targeted_analysis=True,
#     ignore_uncertain_line=True
# )

jtap_metrics = None

jtap_plot_rg_lines(
    jtap_decisions,
    stimulus=jtap_stimulus,
    show="model",
    include_baselines=True,
    include_human=True,
    jtap_metrics=jtap_metrics,
    jtap_run_idx=jtap_run_idx,
    remove_legend=True,
    include_start_frame=True,
    plot_stat="mean",
    include_stimulus=True
)

In [ ]:
jtap_run_idx_viz = 9

rerun_jtap_single_run(JTAPMICE_DATA, rgb_video_highres = None, stimulus_name = "jtap_single_runv3", tracking_dot_size_range = (0.5,2), prediction_line_size_range = (0.05,0.4), jtap_run_idx = jtap_run_idx_viz, grid_dot_radius = 0.3, render_grid = True, show_velocity = True)

In [ ]:
draw_stimulus_image(jtap_stimulus, frame=1)